# Tuner del modello con Keras Tuners

#import e dati

In [13]:
!git clone https://github.com/Jake145/CAE-for-DM-segmentation

Cloning into 'CAE-for-DM-segmentation'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1118 (delta 1), reused 2 (delta 0), pack-reused 1112
Receiving objects: 100% (1118/1118), 1.20 GiB | 32.12 MiB/s, done.
Resolving deltas: 100% (492/492), done.


In [1]:
from google.colab import drive #remember to add the features from the data if you can load it correctly
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install SimpleITK


     |████████████████████████████████| 47.4MB 66kB/s 


In [3]:
!pip install pyradiomics

     |████████████████████████████████| 163kB 15.7MB/s 
     |████████████████████████████████| 112kB 29.8MB/s 
     |████████████████████████████████| 552kB 24.6MB/s 


In [4]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 7.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=28df4eb15ac1af1ea79046e00e68c5fb7bbc3b4ecc4d8a9915d880b68c63536b
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=d14e52d990d7e62c09005627788ae01674f84e0cbdce6ad091f708b3130c0ff0
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [5]:
% pip install CAE-Jake-HP-145

  Created wheel for CAE-Jake-HP-145: filename=CAE_Jake_HP_145-1.4-cp36-none-any.whl size=10829 sha256=a33a45e86a62ec6954eb849ea346d4afbe1c47a08e140ab26e5ee7701bde34b2
  Stored in directory: /root/.cache/pip/wheels/ab/4c/de/8d6f04ed9811f61d750d0f6e36b7001bb097b89dfefe87c41b
Successfully built CAE-Jake-HP-145


In [23]:
import os
from PIL import Image
from skimage.io import imread
import datetime
import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from keras.constraints import unit_norm,min_max_norm,max_norm
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.decomposition import PCA
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
import glob
import radiomics
from radiomics import featureextractor
from sklearn.model_selection import train_test_split
import tensorflow as tf
import kerastuner
from kerastuner.tuners import RandomSearch, BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters
import time
from keras.utils import to_categorical

from functioncae import classes_cae, caehelper
LOG_DIR = f"{int(time.time())}"

# Trasformazioni per data augmentation

In [14]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle

In [15]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip= True,
        fill_mode='reflect')

In [16]:
transform = train_datagen.get_random_transform((124,124)) 
transform



{'brightness': None,
 'channel_shift_intensity': None,
 'flip_horizontal': 0,
 'flip_vertical': 1,
 'shear': -0.18401469594062117,
 'theta': -11.561206522200635,
 'tx': -23.709737246800742,
 'ty': -18.866810923534658,
 'zx': 0.8651383540400697,
 'zy': 1.145654282096864}

#Tuner model. In questo caso utilizziamo la Unet, ma è adattabile a ogni modello

In [18]:
from keras.layers import Conv2D, Conv2DTranspose, Input, Dropout,MaxPooling2D, UpSampling2D, Dense, Flatten
from keras.models import Model, load_model
from keras.layers.experimental.preprocessing import Resizing
from keras.layers.merge import concatenate

def build_model_rad_UNET(hp,shape=(124,124,1),feature_dim=(3,),multilayer=False):
    input_tensor = Input(shape=shape)
    input_vector= Input(shape=feature_dim)

    c1 = Conv2D(16, (3, 3), activation='relu', 
                            padding='same')(input_tensor)
    c1 = Dropout(0.2)(c1)
    c1 = Conv2D(16, (3, 3),activation='relu', 
                            padding='same')(c1)
    p1 =MaxPooling2D((2, 2))(c1)
    c2 = Conv2D(32, (3, 3), activation='relu', 
                            padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', 
                            padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Resizing(32,32,interpolation='nearest')(p2)
    c3 = Conv2D(64, (3, 3), activation='relu', 
                            padding='same')(p2)


    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', 
                            padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Resizing(16,16,interpolation='nearest')(p3)
    c4 = Conv2D(128, (3, 3), activation='relu', 
                            padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', 
                            padding='same')(c4)

    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', 
                            padding='same')(p4)

    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', 
                            padding='same', name="last_conv")(c5)
#fc layers

    flat=Flatten()(c5)
    flat=concatenate([flat,input_vector])
    den = Dense(hp.Int(f'dense_base_unit',
                                min_value=12,
                                max_value=128,
                                step=4), activation='relu')(flat)
    if multilayer:
      for i in range(hp.Int('n_layers', 1, 4)):  # adding variation of layers.


        den = Dense(hp.Int(f'conv_{i}_units',
                                  min_value=4,
                                  max_value=64,
                                  step=4), activation='relu')(den)
        den= Dropout(hp.Float(f'drop_{i}_rate',
                                  min_value=0,
                                  max_value=.5,
                                  step=.1,))(den)

    classification_output = Dense(2, activation = 'sigmoid', name="classification_output")(den)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)

    #c4 = Resizing(14,14,interpolation='nearest')(c4)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', 
                            padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', 
                            padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    #c3= Resizing(28,28,interpolation='nearest')(c3)

    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', 
                            padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', 
                            padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = Resizing(62,62,interpolation='nearest')(c2)

    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', 
                            padding='same')(u8)
    c8 = Dropout(0.2)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', 
                            padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    #c1= Resizing(112,112,interpolation='nearest')(c1)

    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', 
                            padding='same')(u9)
    c9 = Dropout(0.2)(c9)

    c9 = Conv2D(16, (3, 3), activation='relu', 
                            padding='same')(c9)


    decoder_out = Conv2D(1, (1, 1), activation='sigmoid',name="decoder_output")(c9)

    model = Model([input_tensor,input_vector], [decoder_out,classification_output])
    model.compile(optimizer='adam', loss={'decoder_output':'binary_crossentropy','classification_output':'categorical_crossentropy'},
                  metrics={'decoder_output':'MAE','classification_output':tf.keras.metrics.AUC()})
    return model


#Preparazione dati

In [24]:
datapath='/content/CAE-for-DM-segmentation/large_sample_Im_segmented_ref'

X_rad,Y_rad,LABELS_rad = caehelper.read_dataset(datapath,'pgm','_2_resized','_1_resized')
X_rad = X_rad/255
Y_rad = Y_rad/255

x_id ="_resized"
y_id="_mass_mask"
ext='pgm'
fnames = glob.glob(os.path.join(datapath, f"*{x_id}.{ext}"))
fnamesmask = glob.glob(os.path.join(datapath, f"*{y_id}.{ext}"))

extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('gldm')
extractor.enableFeatureClassByName('glcm')
extractor.enableFeatureClassByName('shape2D')
extractor.enableFeatureClassByName('firstorder')
extractor.enableFeatureClassByName('glrlm')
extractor.enableFeatureClassByName('glszm')
extractor.enableFeatureClassByName('ngtdm')
fnames.sort()
dataframe={f.replace(datapath,''):extractor.execute(caehelper.read_pgm_as_sitk(f), caehelper.read_pgm_as_sitk(f.replace(x_id,y_id)),label=255) for f in fnames}


Pandata=pd.DataFrame(dataframe)

Pandataframe=Pandata.drop(Pandata.index[0:22]).T


X_train_rad, X_test_rad, Y_train_rad, Y_test_rad,class_train_rad,class_test_rad,feature_train,feature_test = train_test_split(X_rad, Y_rad,LABELS_rad,Pandataframe, test_size=0.2, random_state=42)



sc = StandardScaler()
feature_train = sc.fit_transform(feature_train)
feature_test = sc.transform(feature_test)


pca = PCA()
feature_train = pca.fit_transform(feature_train)
feature_test = pca.transform(feature_test)
explained_variance = pca.explained_variance_ratio_


pca = PCA(n_components=3)
feature_train = pca.fit_transform(feature_train)
feature_test = pca.transform(feature_test)



train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip= True,
        fill_mode='reflect')

transform = train_datagen.get_random_transform((124,124))


X_train_rad_tr, X_train_rad_val, Y_train_rad_tr, Y_train_rad_val,class_train_rad_tr,class_train_rad_val,feature_train_tr,feature_train_val = train_test_split(X_train_rad, Y_train_rad, to_categorical(class_train_rad,2),feature_train, test_size=0.2, random_state=24)

mass_gen_rad = classes_cae.MassesSequenceRadiomics(X_train_rad_tr, Y_train_rad_tr,class_train_rad_tr,feature_train_tr, train_datagen)




GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

In [ ]:
tuner = BayesianOptimization(
    build_model_rad_UNET,
    objective= kerastuner.Objective("val_classification_output_auc", direction="max"),
    max_trials=10,
    executions_per_trial=2,
    directory=LOG_DIR)

tuner.search(mass_gen_rad,
             verbose=2,
             epochs=100,
             batch_size=len(mass_gen_rad),
             #callbacks=[tensorboard],
             validation_data=([X_train_rad_val,feature_train_val], [Y_train_rad_val,class_train_rad_val]))




Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
dense_base_unit   |68                |?                 

Epoch 1/100
11/11 - 9s - loss: 1.2098 - decoder_output_loss: 0.5335 - classification_output_loss: 0.6763 - decoder_output_MAE: 0.4002 - classification_output_auc: 0.6486 - val_loss: 0.9298 - val_decoder_output_loss: 0.3081 - val_classification_output_loss: 0.6217 - val_decoder_output_MAE: 0.2139 - val_classification_output_auc: 0.7568
Epoch 2/100
11/11 - 0s - loss: 0.8758 - decoder_output_loss: 0.2422 - classification_output_loss: 0.6337 - decoder_output_MAE: 0.0835 - classification_output_auc: 0.7012 - val_loss: 0.7639 - val_decoder_output_loss: 0.1777 - val_classification_output_loss: 0.5862 - val_decoder_output_MAE: 0.0887 - val_classification_output_auc: 0.7996
Epoch 3/100
11/11 - 0s - loss: 0.7494 - decoder_output_loss: 0.1344 - classification_output_loss: 0.6149 - decoder_output_MAE: 0.0545 - classification_output_auc: 0.7217 - val_loss: 0.

In [ ]:
tuner.get_best_hyperparameters()[0].values

newmod=build_model_rad_UNET(tuner.get_best_hyperparameters()[0])
newmod.summary()

tuner.get_best_models()[0].summary()



newmod.save('model_Unet_Tuned') #the model_opt is better
